# Convolutional Neural Networks using Pretrained Model

_**Building an Image Classifier using a Pretrained Xception Model on TensorFlow Flower Dataset.**_

In [ ]:
# Imports required packages

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds    # THIS MIGHT NEED TO BE INSTALLED SEPERATELY
import matplotlib.pyplot as plt

## Retrieving & Analysing Dataset

In [ ]:
# Load the TensorFlow Flowers dataset from specified path
tf_flowers, info = tfds.load("tf_flowers", data_dir="data/tensorflow_datasets", 
                             download=False, as_supervised=True, with_info=True)

In [5]:
# Extract useful information about the dataset

dataset_size = info.splits["train"].num_examples    # Number of instances
class_names = info.features["label"].names          # Name of the flowers
n_classes = info.features["label"].num_classes      # Count of types of flowers

In [ ]:
# Prints basic information about the dataset

print("Size of the dataset:", <code here>)
print("Classes (types of flower):", <code here>)
print("Numer of classes:", <code here>)

In [13]:
# As this dataset does not provide a seperate test dataset, training set gets further splitted
# into test [first 10%], validation [next 15%] and train [remaining 75%] dataset.

test_set_raw, val_set_raw, train_set_raw = tfds.load(
    "tf_flowers",
    split=["train[:10%]", "train[10%:25%]", "train[25%:]"],
    data_dir="data/tensorflow_datasets", 
    download=False, 
    as_supervised=True)

In [ ]:
# Plots any 9 flowers to check how they look

plt.figure(figsize=(12, 10))
index = 0
for image, label in val_set_raw.take(9):
    index += 1
    plt.subplot(3, 3, index)
    plt.imshow(image)
    plt.title(f"Class: {class_names[label]}")
    plt.axis("off")

plt.show()

## Preprocessing

**Resizing Images:** As training instances required to be of the same size during batch processing, image resizing is done over a `Resizing` layer. Function `tf.keras.applications.xception.preprocess_input()` also needs to be called to preprocess the images appropriately for the `Xception` model. Shuffling and prefetching are also added to the training dataset.

In [17]:
tf.random.set_seed(42)

batch_size = 32

# Configures a preprocessing layer for image resizing
preprocess = tf.keras.Sequential([
    # To resize each image
    tf.keras.layers.Resizing(height=224, width=224, crop_to_aspect_ratio=True),

    # to preprocess the resized image appropriately for the Xception model
    tf.keras.layers.Lambda(tf.keras.applications.xception.preprocess_input)
])

# Image instances in different datasets get passed through the preprocessing layer to get resized.

train_set = train_set_raw.map(lambda X, y: (preprocess(X), y))

train_set = train_set.shuffle(1000, seed=42).batch(batch_size).prefetch(1)  # Note that only train dataset gets additional
                                                                            # shuffling for effective batch processing

val_set = val_set_raw.map(lambda X, y: (preprocess(X), y)).batch(batch_size)

test_set = test_set_raw.map(lambda X, y: (preprocess(X), y)).batch(batch_size)

Now each batch contains 32 images, all of them 224 × 224 pixels, with pixel values ranging from –1 to 1.

## Modeling

Loads the pretrained model, without its top layers, and then replace them with the ones specific to this flower classification task.

In [ ]:
# Load the pretrained Xception model for imagenet from the specified path
# consider it a base model

base_model = tf.keras.models.load_model("./models/xception_weights_imagenet.keras")

In [31]:
# Add an global average pooling layer at the output of the base model
avg_pool = <code here>

# On top of the average pooling layer, add a dense output layer specifying output shape and 
# activation function appropriate for classification task
output = <code here>

# Assemble the model considering inputs as base model's input and outputs as 
# output layer initialized in the above step
model = <code here>

**Training the model**

In [73]:
# Freeze all the layers of the base model before training

<code here>

In [ ]:
# Initialize "SGD" as model optimizer with 0.1 as learning rate and 0.9 as momentum
optimizer = <code here>

# Compile the model by specifying sparse categorical crossentropy as loss function,
# already initialized optimizer and "accuracy" as a metric
<code here>

# Fit the model by specifying train set, 1 epoch and validation set
history = <code here>

In [ ]:
# Evaluate the model on test set
<code here>

**Note down the observed accuracy of this model.**

**Observations:**

Note down all your observations in green/blue book.